In [1]:
import os.path as osp
from tqdm import trange, tqdm
from pprint import pprint
import cv2
from pycocotools.coco import COCO
from mmengine import load, dump

# image
*把gray图片转为rgb格式

In [ ]:
import PIL.Image as Image
from pathlib import Path

In [ ]:
def gray2rgb(img_root: Path):
    filenames = sorted(list(img_root.glob('*')))
    error_files = []
    for filename in tqdm(filenames):
        raw = Image.open(filename)
        if len(raw.getbands()) < 3:
            error_files.append(filename)
            rgb = raw.convert('RGB')
            rgb.save(filename)
    print(f'num total image: {len(filenames)}\n'
          f'num gray  image: {len(error_files)}')
    return error_files

In [ ]:
gray2rgb(Path('../../images/test2015'))
gray2rgb(Path('../../images/train2015'))

# annotation

## instance / relation

从HOIA格式，向COCO格式转换

In [64]:
ins_categories = load('../../configs/ins_categories.json')
rel_categories = load('../../configs/rel_categories.json')

In [69]:
def collect_coco_format(raw_anns: list, img_root: str) -> tuple:
    images, ins_annotations, rel_annotations = [], [], []

    for i in trange(len(raw_anns)):
        ann_info = raw_anns[i]
        # 图片信息
        image_id = int(ann_info['file_name'][-12:-4])
        img_path = osp.join(img_root, ann_info['file_name'])
        img = cv2.imread(img_path)  # 原标注的shape可能是错的
        images.append({
            'file_name': ann_info['file_name'],
            'height'   : img.shape[0],
            'width'    : img.shape[1],
            'id'       : image_id
        })

        # 目标检测信息
        ins_ann_ids = []
        for ins_ann in ann_info['annotations']:
            x1, y1, x2, y2 = ins_ann['bbox']
            bbox = [x1, y1, x2-x1, y2-y1]
            ins_ann_id = len(ins_annotations)+1
            ins_ann_ids.append(ins_ann_id)
            ins_annotations.append({
                'id'         : ins_ann_id,
                'bbox'       : bbox,
                'area'       : bbox[-1] * bbox[-2],
                'iscrowd'    : False,
                'category_id': ins_ann['category_id'],
                'image_id'   : image_id,
            })

        # 交互信息
        for rel_ann in ann_info['hoi_annotations']:
            rel_annotations.append({
                'id'         : len(rel_annotations) + 1,
                'image_id'   : image_id,
                'object_id'  : ins_ann_ids[rel_ann[ 'object_id']],
                'subject_id' : ins_ann_ids[rel_ann['subject_id']],
                'category_id': rel_ann['category_id'],
            })

    print(f'num_images: {len(images         )}\n'
          f'num_ins   : {len(ins_annotations)}\n'
          f'num_rel   : {len(rel_annotations)}')
    new_ins_anns = {
        'info'       : None,
        'licenses'   : [],
        'images'     : images,
        'annotations': ins_annotations,
        'categories' : ins_categories
    }
    new_rel_anns = {
        'info'       : None,
        'licenses'   : [],
        'images'     : images,
        'annotations': rel_annotations,
        'categories' : rel_categories
    }
    image_anns = {
        'info'       : None,
        'licenses'   : [],
        'images'     : images,
    }
    return image_anns, new_ins_anns, new_rel_anns

In [ ]:
train_anns = load('../../annotations/raw/train.json')
test_anns  = load('../../annotations/raw/test.json' )
print(f'{len(train_anns)} & {len(test_anns)}')

37633 & 9546


In [ ]:
pprint(train_anns[0], width=100, compact=True, sort_dicts=False)  # 1

{'file_name': 'HICO_train2015_00000001.jpg',
 'img_id': 1,
 'width': 640,
 'height': 480,
 'annotations': [{'bbox': [208, 33, 427, 300], 'category_id': 1},
                 {'bbox': [59, 98, 572, 405], 'category_id': 4},
                 {'bbox': [213, 20, 438, 357], 'category_id': 1},
                 {'bbox': [77, 115, 583, 396], 'category_id': 4},
                 {'bbox': [206, 33, 427, 306], 'category_id': 1},
                 {'bbox': [61, 100, 571, 401], 'category_id': 4},
                 {'bbox': [209, 26, 444, 317], 'category_id': 1},
                 {'bbox': [59, 99, 579, 395], 'category_id': 4}],
 'hoi_annotations': [{'subject_id': 0, 'object_id': 1, 'category_id': 73, 'hoi_category_id': 153},
                     {'subject_id': 2, 'object_id': 3, 'category_id': 77, 'hoi_category_id': 154},
                     {'subject_id': 4, 'object_id': 5, 'category_id': 88, 'hoi_category_id': 155},
                     {'subject_id': 6, 'object_id': 7, 'category_id': 99, 'hoi_categor

In [ ]:
pprint(test_anns[0], width=100, compact=True, sort_dicts=False)  # 1

{'file_name': 'HICO_test2015_00000001.jpg',
 'img_id': 1,
 'width': 640,
 'height': 427,
 'annotations': [{'bbox': [320, 306, 359, 349], 'category_id': 1},
                 {'bbox': [270, 303, 311, 350], 'category_id': 1},
                 {'bbox': [148, 345, 376, 414], 'category_id': 15}],
 'hoi_annotations': [{'subject_id': 0, 'object_id': 2, 'category_id': 88, 'hoi_category_id': 246},
                     {'subject_id': 1, 'object_id': 2, 'category_id': 88, 'hoi_category_id': 246}]}


In [72]:
coco_img_train, coco_ins_train, coco_rel_train = \
    collect_coco_format(train_anns, '../../images/train2015/')
dump(coco_img_train, 'image_info_train.json')
dump(coco_ins_train,  'instances_train.json')
dump(coco_rel_train,  'relations_train.json')

100%|██████████| 37633/37633 [02:56<00:00, 213.72it/s]


num_images    : 37633
num_ins       : 199147
num_rel       : 117870
num_same_rel  : 0


In [70]:
coco_img_test, coco_ins_test, coco_rel_test = \
    collect_coco_format(test_anns, '../../images/test2015')
dump(coco_img_test, 'image_info_test.json')
dump(coco_ins_test,  'instances_test.json')
dump(coco_rel_test,  'relations_test.json')

100%|██████████| 9546/9546 [00:43<00:00, 221.68it/s]


num_images    : 9546
num_ins       : 56786
num_rel       : 33405
num_same_rel  : 0


QPIC提供的标注和原版略有不同，对box进行了些许merge，导致其和pasta标注不兼容

In [ ]:
train_anns = load('../../annotations/qpic/train_qpic.json')
test_anns  = load('../../annotations/qpic/test_qpic.json' )
print(f'{len(train_anns)} & {len(test_anns)}')

In [ ]:
coco_img_train, coco_ins_train, coco_rel_train = \
    collect_coco_format(train_anns, '../../images/train2015/')
dump(coco_img_train, 'image_info_train_qpic.json')
dump(coco_ins_train,  'instances_train_qpic.json')
dump(coco_rel_train,  'relations_train_qpic.json')

In [ ]:
coco_img_test, coco_ins_test, coco_rel_test = \
    collect_coco_format(test_anns, '../../images/test2015')
dump(coco_img_test, 'image_info_test_qpic.json')
dump(coco_ins_test,  'instances_test_qpic.json')
dump(coco_rel_test,  'relations_test_qpic.json')

## pasta

由hake数据集的标注转换而来

In [2]:
from utils import box_xyxy2xywh

In [3]:
coco_ins_train = COCO('../../annotations/instances_train.json')
coco_rel_train = COCO('../../annotations/relations_train.json')

loading annotations into memory...
Done (t=0.68s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [4]:
pasta_categories = load('../../configs/pasta_categories.json')

In [5]:
hake_anns = load('../../annotations/raw/hake_large_annotation.json')

In [6]:
pprint(coco_ins_train.imgToAnns[1], compact=True, width=120)

[{'area': 58473, 'bbox': [208, 33, 219, 267], 'category_id': 1, 'id': 1, 'image_id': 1, 'iscrowd': False},
 {'area': 157491, 'bbox': [59, 98, 513, 307], 'category_id': 4, 'id': 2, 'image_id': 1, 'iscrowd': False},
 {'area': 75825, 'bbox': [213, 20, 225, 337], 'category_id': 1, 'id': 3, 'image_id': 1, 'iscrowd': False},
 {'area': 142186, 'bbox': [77, 115, 506, 281], 'category_id': 4, 'id': 4, 'image_id': 1, 'iscrowd': False},
 {'area': 60333, 'bbox': [206, 33, 221, 273], 'category_id': 1, 'id': 5, 'image_id': 1, 'iscrowd': False},
 {'area': 153510, 'bbox': [61, 100, 510, 301], 'category_id': 4, 'id': 6, 'image_id': 1, 'iscrowd': False},
 {'area': 68385, 'bbox': [209, 26, 235, 291], 'category_id': 1, 'id': 7, 'image_id': 1, 'iscrowd': False},
 {'area': 153920, 'bbox': [59, 99, 520, 296], 'category_id': 4, 'id': 8, 'image_id': 1, 'iscrowd': False}]


In [7]:
coco_rel_train.dataset['annotations'][0]

{'id': 1, 'subject_id': 1, 'object_id': 2, 'category_id': 73, 'image_id': 1}

In [8]:
pprint(hake_anns['HICO_train2015_00000526.jpg'], compact=True)

{'dataset': 'hico-det',
 'labels': [{'action_labels': [{'human_part': 1, 'partstate': 5},
                               {'human_part': 4, 'partstate': 0},
                               {'human_part': 5, 'partstate': 4},
                               {'human_part': 0, 'partstate': 0},
                               {'human_part': 1, 'partstate': 5},
                               {'human_part': 7, 'partstate': 1},
                               {'human_part': 8, 'partstate': 1}],
             'height': 480,
             'hoi_id': 94,
             'human_bbox': [1, 98, 616, 474],
             'object_bbox': [1, 2, 634, 384],
             'width': 640}],
 'path_prefix': 'hico_20160224_det/images/train2015'}


In [57]:
def collect_coco_pasta(hake_anns: list, coco_ins: COCO, coco_rel: COCO) -> dict:
    imgs_with_same_pasta = set()
    pasta_annotations = []

    for image_id, rel_info in tqdm(coco_rel.imgToAnns.items()):
        ins_info = coco_ins.imgToAnns[image_id]
        hake_info = hake_anns[coco_rel.imgs[image_id]['file_name']]

        valid_bboxes = [tuple(item['bbox']) for item in ins_info]
        box_pair_2_rel_ann_id = {
            (
                tuple(coco_ins.anns[rel['subject_id']]['bbox']), 
                tuple(coco_ins.anns[rel['object_id']]['bbox'])
            ): rel['id']
            for rel in rel_info}

        # part state信息
        check_same = set()
        for ann in hake_info['labels']:
            sub_bbox = tuple(box_xyxy2xywh(ann[ 'human_bbox']))
            obj_bbox = tuple(box_xyxy2xywh(ann['object_bbox']))
            assert sub_bbox in valid_bboxes
            assert obj_bbox in valid_bboxes

            try:
                rel_ann_id = box_pair_2_rel_ann_id[(sub_bbox, obj_bbox)]
            except KeyError as err:
                print(image_id)
                raise err

            for pasta in ann.get('action_labels', []):
                part_id = pasta['human_part']
                state_id = pasta['partstate']
                category_id = f'{part_id}-{state_id}'

                tmp = (rel_ann_id, category_id)
                if tmp in check_same:
                    imgs_with_same_pasta.add(image_id)
                    continue
                else:
                    check_same.add(tmp)

                pasta_annotations.append({
                    'id'         : len(pasta_annotations)+1,
                    'image_id'   : image_id,
                    'rel_ann_id' : rel_ann_id,
                    'category_id': category_id
                })

    print('num pasta     :', len(pasta_annotations   ))
    print('num same_pasta:', len(imgs_with_same_pasta))
    coco_pasta = {
        'info'       : coco_rel.dataset['info'    ],
        'licenses'   : coco_rel.dataset['licenses'],
        'images'     : coco_rel.dataset['images'  ],
        'annotations': pasta_annotations,
        'categories' : pasta_categories
    }
    return coco_pasta, sorted(list(imgs_with_same_pasta))

In [58]:
coco_pasta_train, imgs_with_same_pasta = collect_coco_pasta(
    hake_anns, coco_ins_train, coco_rel_train)

100%|██████████| 37633/37633 [00:00<00:00, 71592.61it/s]

num pasta     : 224308
num same_pasta: 20


In [ ]:
dump(coco_pasta_train, 'pasta_train.json')